## Gaussian Elimination

In this section we develop Python code to solve linear systems in the most direct way, via **Gaussian Elimination** (or just **elimination**).

**The main concept of elimination:**

* transform the given system to an equivalent system (with same solution) that but that is *easier* to solve.

The transformation comes in the form of a sequence of **row operations** that preserve the solution of the system while making the solution readily available. There are three row operations that are helpful:

1. Exchange the position of two rows/equations.
    * Notation: $R_i \xleftrightarrow{} R_j$
4. Multiply a row/equation by any nonzero number:
    * Notation: $R_i \to k R_i$
6. Replace any row/equation with the sum of itself and a multiple of another row/equation.
   * Notation: $R_i \to R_i + k R_j$



<a id='GE1'></a>
### Example:  Row operations and elimination

Let's look at an example.

$$
\begin{eqnarray*}
x + 2 y - z & = & 3\\
2x + y - 2z & = & 3\\
-3x + y + z & = & -6
\end{eqnarray*}
$$

Written in augmented form:

$$
\begin{equation}
\left[ \begin{array}{rrr|r}
1 & 2 & -1 & 3 \\
2 & 1 & -2 & 3 \\
-3 & 1 & 1 & -6
\end{array}\right]
\end{equation}
$$

Row operations to transform to an equivalent (upper triangular matrix) are

1. $R_2 \to R_2 - 2 R_1$
2. $R_3 \to R_3 + 3 R_1$
3. $R_3 \to R_3 + \dfrac{7}{3} R_2$

and results in the follwoing equivalent system

$$
\begin{equation}
\left[ \begin{array}{rrr|r}
1 & 2 & -1 & 3 \\
0 & -3 & 0 & -3 \\
0 & 0 & -2 & -4
\end{array}\right]
\end{equation}
$$

At this point, back-substitution can be used to arrive at the solution. Rather than do this now, let's build a python routine to execute the elimination process, illustrated above, to a more general $n\times n$ system.  

We'll assign the array the name $A$, so that we can refer to it later.

In [11]:
import sys
import numpy as np
# Define the augmented matrix in the above example.
# We'll use this as a test case for working with nxn systems.
A = np.array([[1,2,-1,3],[2,1,-2,3],[-3,1,1,-6]])
print(A)

[[ 1  2 -1  3]
 [ 2  1 -2  3]
 [-3  1  1 -6]]


We could start performing operations on our array, but instead we will first write a few bits of code that will do each of these operations in a systematic manner.


The goal of elimination is to perform row operations to produce a an equivalent upper-triangular matrix:

$$
\begin{equation}
\left[ \begin{array}{cccc} 1 & * & * & * \\ 0 & 1 & * & * \\ 0 & 0 & 1 & * \end{array}\right]
\end{equation}
$$

where the asterisk denote some entry (may or may not be zero). Let's now generalize this to an $n\times n$ system:  

In [19]:
def elimin(A,b):
# =============================================================================
#     A is an nxn matrix .
#     b is the right-hand-side of Ax = b.
#     indicies counting from 1. Note python uses zero-based indexing
# =============================================================================
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A

    if m != n:
        print("error! aatrix A must be a square matrix (n x n)")
        return None, None

    x = np.zeros(n)

    B = np.copy(A).astype('float64')
    r = np.copy(b).astype('float64')
    eps = sys.float_info.epsilon

    # visit each column, up to j = n-1
    for j in range(n-1):
        if abs(B[j][j]) < eps:
            print("error: 0 pivot encountered")
            return None, None

      # Iterate through each row below the pivot row
        for i in range(j + 1, n):
            scale = B[i][j] / B[j][j]
            for k in range(j, n):
                B[i][k] = B[i][k] - scale * B[j][k]

            r[i] = r[i] - scale * r[j]

    # === Back-Substitution ===
    for i in range(n - 1, -1, -1):
        dot_sum = 0
        for k in range(i + 1, n):
            dot_sum += B[i][k] * x[k]

        if abs(B[i][i]) < eps:
            print("error: division by 0 during back subs")
            return None, None

        x[i] = (r[i] - dot_sum) / B[i][i]

    return B, x


In [13]:
elimin(A[0:3,0:3],A[0:3,3])

(array([[ 1.,  2., -1.],
        [ 0., -3.,  0.],
        [ 0.,  0., -2.]]),
 array([3., 1., 2.]))

# ai


In [14]:
A_a1 = np.array([[2, -3],
                 [5, -6]])
b_a1 = np.array([2, 8])

U_a1, x_a1 = elimin(A_a1, b_a1)

print("Sol for a(i):")
print(x_a1)

Sol for a(i):
[4. 2.]


aii

In [15]:
A_a2 = np.array([[1, 2, -1],
                 [0, 3, 1],
                 [2, -1, 1]])
b_a2 = np.array([2, 4, 2])

U_a2, x_a2 = elimin(A_a2, b_a2)

print("Sol for a(ii):")
print(x_a2)

Sol for a(ii):
[1. 1. 1.]


b

In [20]:
A_b = np.array([[1, -1, 2, 0],
                [0, 2, 1, 0],
                [1, 3, 4, 4],
                [4, 2, 1, -1]])
b_b = np.array([1, 2, 3, 4])

U_b, x_b = elimin(A_b, b_b)

print("Sol for b:")
print(x_b)

error: 0 pivot encountered
Sol for b:
None


5)

In [21]:
def lu_decomposition(A):
    m, n = A.shape

    # 1. Validation: Ensure the matrix is square.
    if m != n:
        print("error: matrix must be square for LU decomposition")
        return None, None

    # 2. Initialization
    U = np.copy(A).astype('float64')
    L = np.identity(n).astype('float64')
    eps = sys.float_info.epsilon

    for j in range(n - 1):
        if abs(U[j, j]) < eps:
            print(f"error: 0 pivot encountered at position ({j},{j})")
            return None, None

        for i in range(j + 1, n):
            scale = U[i, j] / U[j, j]
            L[i, j] = scale
            U[i, :] = U[i, :] - scale * U[j, :]

    return L, U